# Degradation and Acceleration Factors
This tool will provide a simple method for estimating degradation and for calculating acceleration factors. It interfaces with the degradation database to simplify acquisition of the degradation parameters.

**Requirements**:
- compatible weather file (e.g., PSM3, TMY3, EPW...)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
    - & etc.
- Activation energies for test material [kJ/mol]
- Other degradation parameters

**Objectives**:
1. Read in the weather data
2. Gather basic degradation modeling data for a material of interest
3. Calculate absolute degradation rate
4. Run Monte Carlo simulation at a single site
5. Generate chamber or field data for environmental comparison
6. Calculate degradation acceleration factor of field location to chamber (or another location)
7. Run Monte Carlo simulation of the acceleration factor for a single site
8. Select a region of interest and specific data points
9. Produce a map of acceleration factors for a region

In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.4.2

In [5]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
import numpy as np
import json
import pvlib

NameError: name 'jit' is not defined

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)
print(DATA_DIR)

Working on a  Windows 10
Python version  3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.4
pvdeg version  0.4.3.dev14+g08d739a
C:\Users\mkempe\Documents\GitHub\new\PVDegradationTools\pvdeg\data


## 1. Read In the Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [2]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
weather_df, meta = pvdeg.weather.read(weather_file,'csv')
print(weather_file)
print(meta)

C:\Users\mkempe\Documents\GitHub\new\PVDegradationTools\pvdeg\data\psm3_demo.csv
{'Source': 'NSRDB', 'Location ID': 145809.0, 'City': 'West Pleasant View', 'State': 'Colorado', 'Country': 'United States', 'Clearsky DHI Units': 'w/m2', 'Clearsky DNI Units': 'w/m2', 'Clearsky GHI Units': 'w/m2', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Solar Zenith Angle Units': 'Degree', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Relative Humidity Units': '%', 'Precipitable Water Units': 'cm', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Cloud Type -15': 'N/A', 'Cloud Type 0': 'Clear', 'Cloud Type 1': 'Probably Clear', 'Cloud Type 2': 'Fog', 'Cloud Type 3': 'Water', 'Cloud Type 4': 'Super-Cooled Water', 'Cloud Type 5': 'Mixed', 'Cloud Type 6': 'Opaque Ice', 'Cloud Type 7': 'Cirrus', 'Cloud Type 8': 'Overlapping', 'Cloud Type 9': 'Overshooting', 'Cloud Type 10': 'Unknown', 'Cloud Type 11': 'Dust', 'Cloud Type 12': 'Smoke', 'Fill F

In [ ]:
# This routine will get a meteorological dataset from anywhere in the world where it is available 
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
#weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
weather_id = (33.4152, -111.8315) #Mesa, Arizona
#weather_id = (0,0) # Somewhere else you are interested in.
weather_df, meta = pvdeg.weather.get_anywhere(id=weather_id)
print(meta)
#display(weather_df)

{'Source': 'NSRDB', 'Location ID': '77855', 'City': 'Mesa', 'State': 'Arizona', 'Country': 'United States', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Surface Albedo Units': 'N/A', 'Version': '3.2.0', 'latitude': 33.41, 'longitude': -111.82, 'altitude': 381, 'tz': -7, 'wind_height': 2, 'house_number': '501', 'retail': 'Old West Plaza', 'ISO3166-2-lvl4': 'US-AZ', 'Street': 'East 2nd Avenue', 'County': 'Maricopa County', 'Zipcode': '85204', 'Country Code': 'US'}


#### POA Irradiance
Next we need to calculate the stress parameters including temperature and humidity. We start with POA irradiance.
Irradiance_kwarg governs the array orientation for doing the POA calculations. 
It is defaulted to a north-south single axis tracking. A fixed tilt set of parameters is included but is blocked out. 
Look in spectral.py and/or PVLib here for 1-axis kwargs, 
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.tracking.singleaxis.html#pvlib.tracking.singleaxis  
and for fixed tilt,  
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.irradiance.gti_dirint.html?highlight=poa .  

In [ ]:
#irradiance_kwarg ={
    #"tilt": None,
    #"azimuth": None,
    #"module_mount": 'fixed'}
irradiance_kwarg ={
    "axis_tilt": None,
    "axis_azimuth": None,
    "module_mount": '1_axis'}
poa_df = pvdeg.spectral.poa_irradiance(weather_df=weather_df, meta=meta, **irradiance_kwarg)
#display(poa_df)

The array axis_azimuth was not provided, therefore an azimuth of 180.0 was used.
The array axis_tilt was not provided, therefore an axis tilt of 0° was used.



#### Get Spectrally Resolved Irradiance Data
This first set of commands will calculate spectrally resolved irradiance data. This may or may not be needed for a given degradation model and can be skipped here.

In [3]:
# this whole block needs to be replaced with call to calculate spectrally resolved irradiance.

from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
#display(data)
print(INPUT_SPECTRA)


NameError: name 'jit' is not defined

#### Cell or Module Surface Temperature
The following will calculate the cell and module surface temperature using the King model as a default. Other models can be used as described at,  
https://pvlib-python.readthedocs.io/en/stable/reference/pv_modeling/temperature.html. The difference is less than one °C for ground mounted systems
but can be as high as 3 °C for a high temperature building integrated system.

Here the temperatures are added to the dataframe and the module temperature is selected as the default 'temperatue' for the degradation calculations. If it is a cell degradation that is being investigated, temp_cell should be used.

In [4]:
temp_cell = pvdeg.temperature.cell(weather_df=weather_df, meta=meta, poa=poa_df)
temp_module = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_df)

weather_df['temp_cell'] = temp_cell
weather_df['temp_module'] = temp_module
weather_df['temperature'] = weather_df['temp_module']

NameError: name 'pvdeg' is not defined

#### Humidity
Depending on the component for which the calculation is being run on, the desired humidity may be the atmospheric humidity, the module surface humidity, the humidity in front of a cell with a permeable backsheet, the humidity in the backsheet, the humidity in the back encapsulant or another custom humidity location such as a diffusion limited location. The folowing are options for doing all of these calculations. Here all the different humidities are put in the weather_df dataframe, but to select one to be specifically used it should be named 'RH' for most degradation functions (check the documentation of a specific degradation calculation if in doubt). Here the surface humidity is selected as a default.

In [6]:
RH_surface = pvdeg.humidity.surface_outside(weather_df['relative_humidity'], weather_df['temp_air'],temp_module)
RH_front_encapsulant = pvdeg.humidity.front_encap(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,encapsulant='W001')
#display(weather_df)
#display(RH_surface)
#display(temp_module)


Ce_back_encapsulant = pvdeg.humidity.Ce(
    temp_module = temp_module, 
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001')

display(type(Ce_back_encapsulant))
print(Ce_back_encapsulant)
display(Ce_back_encapsulant)
display(RH_front_encapsulant)
#RH_back_encapsulant = pvdeg.humidity.Ce(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,backsheet='W017',encapsulant='W001',output='rh')
#RH_backsheet = (RH_surface + RH_back_encapsulant) / 2

weather_df['RH_surface'] = RH_surface
weather_df['RH_front_encapsulant'] = RH_front_encapsulant
weather_df['Ce_back_encapsulant'] = Ce_back_encapsulant   
#weather_df['RH_back_encapsulant'] = RH_back_encapsulant
#weather_df['RH_backsheet'] = RH_backsheet

weather_df['RH'] = weather_df['RH_surface']

display(weather_df)



AttributeError: 'UndefinedType' object has no attribute 'name'

In [15]:
display(Ce_back_encapsulant.type())


AttributeError: 'DataFrame' object has no attribute 'type'

Each of the necessary arrays of data can be individually sent to a function for calculation in the function call, or they can be combined into a single dataframe. The degradation functions are set up to first check for a specific data set in the function call but if not found it looks for specific data or a suitable substitute in the weather dataframe.

Either the cell temperature or the module temperature can be used. 
If the model needs wavelength resolved spectra, that should be added as 'irradiance'. But for just intensity, the 'poa_global'

In [ ]:
#weather_df['temperature'] = temp_cell
weather_df['temperature'] = temp_module
weather_df['poa_global'] = poa_df['poa_global']


## 2. Gather Basic Degradation Modeling Data for a Material of Interest

First we need to gather in the parameters for the degradation process of interest. This includes things such as the activiation energy and parameters defining the sensitivity to moisture, UV light, voltage, and other stressors.
For this tutorial we will need solar position, POA, PV cell and module temperature. Let's gernate those individually with their respective functions.
The blocked out text will produce a list of key fields from the database for each entry.

In [10]:
#kwarg_variables = pvdeg.utilities._read_material(name=None, fname="DegradationDatabase", item=("Material", "Equation", "KeyWords", "EquationType"))
#print(json.dumps(kwarg_variables, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('},' + "\n", '},' +'\n' +'\n'))
pvdeg.utilities.display_json(pvdeg_file="DegradationDatabase", fp=DATA_DIR)

This next set of codes will take the data from the extracted portion of the Json library and create a list of variables from it. If more variables need to be modified or added, this is where it should be done.

In [15]:
deg_data = pvdeg.utilities.read_material(fp=DATA_DIR, key="D036", pvdeg_file="DegradationDatabase")
print(json.dumps(deg_data, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('\n'+'}', '}'))

{"DataEntryPerson": "Michael Kempe",
"DateEntered": "2/14/2025",
"DOI": "10.1109/PVSC45281.2020.9300357",
"SourceTitle": "Highly Accelerated UV Stress Testing for Transparent Flexible Frontsheets",
"Authors": "Michael D Kempe, Peter Hacke, Joshua Morse, Michael Owen-Bellini, Derek Holsapple, Trevor Lockman, Samantha Hoang, David Okawa, Tamir Lance, Hoi Hong Ng",
"Reference": "Kempe, M. D., et al. (2020). Highly Accelerated UV Stress Testing for Transparent Flexible Frontsheets. 2020 47th IEEE Photovoltaic Specialists Conference (PVSC).",
"KeyWords": "Humidity, Irradiance, reciprocity",
"Material": "Flexible Frontsheet, Frontsheet Coatings",
"Degradation": "UV Cut On, UV Transmittance 310nm-350nm, Yellowness index, SPQEWT",
"EquationType": "arrhenius",
"Equation": "R_D=R_0*RH^n*G_340^P*e^(-E_a/K_(b*T_K ) )",
"R_D":
{"units": "%/h"},
"R_0":
{"units": "%/h"},
"E_a":
{"value": 38.7,
"stdev": 21.7,
"units": "kJ/mol"},
"n":
{},
"p":
{"value": 0.49,
"stdev": 0.22},
"Corr-E_a-P":
{"value": -0.

Here we pull out the relevant variables and equation code identifier needed for running the calculations.

In [8]:
func = "pvdeg.degradation." + deg_data["EquationType"]
print(func)


pvdeg.degradation.arrhenius


## 3. Calculate Absolute Degradation Rate

To do this calculation, we must have degradation parameter data for a process that is complete with all the necessary variables. 

In [26]:
from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
display(data)
print(INPUT_SPECTRA)



,Unnamed: 0,RH,Temperature,"Spectra: [ 300, 325, 350, 375, 400 ]"
0,2021-01-01 00:00:00-05:00,95,-1.1,"[nan, nan, nan, nan, nan]"
1,2021-01-01 01:00:00-05:00,72,-1.1,"[nan, nan, nan, nan, nan]"
2,2021-01-01 02:00:00-05:00,72,-1.3,"[nan, nan, nan, nan, nan]"
3,2021-01-01 03:00:00-05:00,72,-1.5,"[nan, nan, nan, nan, nan]"
4,2021-01-01 04:00:00-05:00,72,-1.7,"[nan, nan, nan, nan, nan]"
...,...,...,...,...
8755,2021-12-31 19:00:00-05:00,92,-1.1,"[nan, nan, nan, nan, nan]"
8756,2021-12-31 20:00:00-05:00,92,-0.7,"[nan, nan, nan, nan, nan]"
8757,2021-12-31 21:00:00-05:00,92,-0.4,"[nan, nan, nan, nan, nan]"
8758,2021-12-31 22:00:00-05:00,92,0.0,"[nan, nan, nan, nan, nan]"


C:\Users\mkempe\Documents\GitHub\new\PVDegradationTools\tests\data\spectra_pytest.csv


                     Unnamed: 0  RH  Temp  \
0     2021-01-01 00:00:00-05:00  95  -1.1   
1     2021-01-01 01:00:00-05:00  72  -1.1   
2     2021-01-01 02:00:00-05:00  72  -1.3   
3     2021-01-01 03:00:00-05:00  72  -1.5   
4     2021-01-01 04:00:00-05:00  72  -1.7   
...                         ...  ..   ...   
8755  2021-12-31 19:00:00-05:00  92  -1.1   
8756  2021-12-31 20:00:00-05:00  92  -0.7   
8757  2021-12-31 21:00:00-05:00  92  -0.4   
8758  2021-12-31 22:00:00-05:00  92   0.0   
8759  2021-12-31 23:00:00-05:00  93  -0.6   

     Spectra: [ 300, 325, 350, 375, 400, ]  
0                [nan, nan, nan, nan, nan]  
1                [nan, nan, nan, nan, nan]  
2                [nan, nan, nan, nan, nan]  
3                [nan, nan, nan, nan, nan]  
4                [nan, nan, nan, nan, nan]  
...                                    ...  
8755             [nan, nan, nan, nan, nan]  
8756             [nan, nan, nan, nan, nan]  
8757             [nan, nan, nan, nan, nan]  
8758     

## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

As with most `pvdeg` functions, the following functions will always require two arguments (weather_df and meta)

In [9]:
# chamber irradiance (W/m^2)
I_chamber = 1000
# chamber temperature (C)
temp_chamber = 60

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber,
                                              poa=poa_df,
                                              temp=temp_cell)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META,
                                                   poa=poa_df,
                                                   temp=temp_cell)

NameError: name 'WEATHER' is not defined

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Equation:
$$ AF = N * \frac{ I_{chamber}^x * RH_{chamber}^n * e^{\frac{- E_a}{k T_{chamber}}} }{ \Sigma (I_{POA}^x * RH_{outdoor}^n * e^{\frac{-E_a}{k T_outdoor}}) }$$

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

Equation:
$$ I_{WA} = [ \frac{ \Sigma (I_{outdoor}^x * RH_{outdoor}^n e^{\frac{-E_a}{k T_{outdood}}}) }{ N * RH_{WA}^n * e^{- \frac{E_a}{k T_eq}} } ]^{\frac{1}{x}} $$

In [26]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                poa=poa_df,
                                                temp=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    poa=poa_df,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    temp=temp_cell,
                                                    Ea=Ea)

## 5. Quick Method (Degradation)

For quick calculations, you can omit POA and both module and cell temperature. The function will calculate these figures as needed using the available weather data with the default options for PV module configuration.

In [27]:
# chamber settings
I_chamber= 1000
temp_chamber=60
rh_chamber=15

# activation energy
Ea = 40

vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=WEATHER, meta=META,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber)

irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=WEATHER, meta=META)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


In [28]:
rh_surface = pvdeg.humidity.surface_outside(rh_ambient=WEATHER['relative_humidity'],
                                               temp_ambient=WEATHER['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=WEATHER, meta=META,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=WEATHER, meta=META,
                                                    rh_outdoor=WEATHER['relative_humidity'],
                                                    Ea=Ea)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


## 6. Solder Fatigue

Estimate the thermomechanical fatigue of flat plate photovoltaic module solder joints over the time range given using estimated cell temperature. Like other `pvdeg` funcitons, the minimal parameters are (weather_df, meta). Running the function with only these two inputs will use default PV module configurations ( open_rack_glass_polymer ) and the 'sapm' temperature model over the entire length of the weather data. 

In [29]:
fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.


If you wish to reduce the span of time or use a non-default temperature model, you may specify the parameters manually. Let's try an explicit example.
We want the solder fatigue estimated over the month of June for a roof mounted glass-front polymer-back module.

1. Lets create a datetime-index for the month of June.
2. Next, generate the cell temperature. Make sure to explicity restrict the weather data to our dt-index for June. Next, declare the PV module configuration.
3. Calculate the fatigue. Explicity specify the time_range (our dt-index for June from step 1) and the cell temperature as we caculated in step 2

In [30]:
# select the month of June
time_range = WEATHER.index[WEATHER.index.month == 6]

# calculate cell temperature over our selected date-time range.
# specify the module configuration
temp_cell = pvdeg.temperature.cell(weather_df=WEATHER.loc[time_range], meta=META,
                                   temp_model='sapm',
                                   conf='insulated_back_glass_polymer')


fatigue = pvdeg.fatigue.solder_fatigue(weather_df=WEATHER, meta=META,
                                       time_range = time_range,
                                       temp_cell = temp_cell)

The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
